In [ ]:
import airpy as ap
import numpy as np
import pandas as pd
import gensim
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim import models
import pyLDAvis.gensim
import csv
import feather
import random

In [ ]:
#get common names list from disk
path_names = '~/data/first_message_text_names.csv'
common_names = list(pd.read_csv(path_names)['dim_first_name'])

#get message text from disk
path_messages = '~/data/first_message_text_lower.csv'
df1 = pd.DataFrame(pd.read_csv(path_messages)['message_text_lower'].dropna())

In [ ]:
#build documents list (remember to drop na's)
documents = []
for index, row in df1.iterrows():
    documents.append(row.message_text_lower)
    
#generate a stoplist
stoplist = stopwords.words('english')
stoplist += common_names

print 'common names done'

# tokenize the text 
doc_token = [[word for word in list(gensim.utils.tokenize(document)) if word not in stoplist] for document in documents]

print 'tokenization done'

# build it into dictionary map
dictionary = corpora.Dictionary(doc_token)

# create the corpus for LDA 
corpus = [dictionary.doc2bow(message_text) for message_text in list(doc_token)]

# convert to TD-IDF
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]


In [ ]:
# train a LDA model
%time 

lda_model_30 = gensim.models.LdaModel(corpus_tfidf, num_topics=30, alpha=0.0001, id2word=dictionary, passes=6)
lda_model_30.save('lda_test.model')

In [ ]:
lda_model_30 = models.LdaModel.load('lda_test.model')
cx_data =  pyLDAvis.gensim.prepare(lda_model_30, corpus_tfidf, dictionary)
pyLDAvis.display(cx_data)

In [ ]:
#make a list of message text, what topic was the most likely for that message text

random.seed(20170710)
#sample 10000 random contacts
n_corpus = len(corpus)
n_sample = 10000

mask = random.sample(xrange(n_corpus), n_sample)
messages_topics = []

#find the most relevant topic to each of these documents
for i in range(n_sample):
    #get array of topics and probabilities for document i
    results_i = lda_model_30[corpus[mask[i]]]
    
    ret = results_i[[item[1] for item in results_i].index(max([item[1] for item in results_i]))]
    
    messages_topics.append([documents[mask[i]], ret[0], ret[1]])
    
    
messages_topic_0 = [x for x in messages_topics if x[1] == 0]
messages_topic_1 = [x for x in messages_topics if x[1] == 1]
messages_topic_2 = [x for x in messages_topics if x[1] == 2]
messages_topic_3 = [x for x in messages_topics if x[1] == 3]
messages_topic_4 = [x for x in messages_topics if x[1] == 4]
messages_topic_5 = [x for x in messages_topics if x[1] == 5]

    